In [1]:
# import libraries
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns 
import html
from bs4 import BeautifulSoup
import requests
import os
from selenium import webdriver
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import warnings; warnings.simplefilter('ignore')
from VE_scraper_functions import *
from chromedriver_py import binary_path # this will get you the path variable
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Daniel.Lang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
path = "pyvpages.xlsx"
park_scrape_dataset2 = pd.read_excel(path)
park_scrape_dataset2.head

<bound method NDFrame.head of        park                                                url
0      abli                 /abli/planyourvisit/lincolnbio.htm
1      abli                      /abli/planyourvisit/hours.htm
2      abli               /abli/planyourvisit/boyhood-home.htm
3      abli                  /abli/planyourvisit/basicinfo.htm
4      abli                 /abli/planyourvisit/directions.htm
...     ...                                                ...
25355  zion       /zion/planyourvisit/laverkin-creek-trail.htm
25356  zion                  /zion/planyourvisit/lightning.htm
25357  zion         /zion/planyourvisit/other-special-uses.htm
25358  zion      /zion/planyourvisit/park-pass-information.htm
25359  zion  /zion/planyourvisit/wildcat-canyon-northgate-p...

[25360 rows x 2 columns]>

In [6]:
website_stems = park_scrape_dataset2[['park', 'url']]
website_stems.head

<bound method NDFrame.head of        park                                                url
0      abli                 /abli/planyourvisit/lincolnbio.htm
1      abli                      /abli/planyourvisit/hours.htm
2      abli               /abli/planyourvisit/boyhood-home.htm
3      abli                  /abli/planyourvisit/basicinfo.htm
4      abli                 /abli/planyourvisit/directions.htm
...     ...                                                ...
25355  zion       /zion/planyourvisit/laverkin-creek-trail.htm
25356  zion                  /zion/planyourvisit/lightning.htm
25357  zion         /zion/planyourvisit/other-special-uses.htm
25358  zion      /zion/planyourvisit/park-pass-information.htm
25359  zion  /zion/planyourvisit/wildcat-canyon-northgate-p...

[25360 rows x 2 columns]>

In [7]:
def extract_text(url):
    start = url.find('planyourvisit/') + len('planyourvisit/')
    end = url.find('.htm', start)
    if start != -1 and end != -1:
        return url[start:end]
    return None

park_scrape_dataset2['url_stem'] = park_scrape_dataset2['url'].apply(extract_text)
park_scrape_dataset2.head

<bound method NDFrame.head of        park                                                url                               url_stem
0      abli                 /abli/planyourvisit/lincolnbio.htm                             lincolnbio
1      abli                      /abli/planyourvisit/hours.htm                                  hours
2      abli               /abli/planyourvisit/boyhood-home.htm                           boyhood-home
3      abli                  /abli/planyourvisit/basicinfo.htm                              basicinfo
4      abli                 /abli/planyourvisit/directions.htm                             directions
...     ...                                                ...                                    ...
25355  zion       /zion/planyourvisit/laverkin-creek-trail.htm                   laverkin-creek-trail
25356  zion                  /zion/planyourvisit/lightning.htm                              lightning
25357  zion         /zion/planyourvisit/other-specia

In [8]:
value_counts = park_scrape_dataset2['url_stem'].value_counts()
most_common = value_counts[value_counts > 100]
print(most_common)

basicinfo                 926
conditions                904
maps                      893
index                     815
fees                      809
directions                789
calendar                  768
things2do                 764
hours                     747
accessibility             457
pets                      428
safety                    420
weather                   335
permitsandreservations    290
placestogo                284
eatingsleeping            248
nearbyattractions         216
visitorcenters            189
guidedtours               177
camping                   170
hiking                    170
outdooractivities         167
fishing                   164
lodging                   142
gettingaround             121
brochures                 121
Name: url_stem, dtype: int64


In [9]:
# Define accessibility words
accessibility_words = [
    "wheelchair", "disability", "impaired", "disabilities",
    "handicap", " ada ", "braille", "disabled", "handicapped", "access", "accessibility"
]

# Function to count accessibility words in text
def count_accessibility_words(text):
    word_counts = {word: text.lower().count(word) for word in accessibility_words}
    return word_counts

# Apply the function to 'url_stem' column and convert to DataFrame
word_counts_df = park_scrape_dataset2['url_stem'].apply(count_accessibility_words).apply(pd.Series)
word_counts_df['access_page'] = word_counts_df.apply(lambda row: 1 if row.any() >= 1 else 0, axis=1)
word_counts_df = word_counts_df[['access_page']]

# Display the resulting DataFrame
print(word_counts_df.head())

   access_page
0            0
1            0
2            0
3            0
4            0


In [11]:
park_scrape_dataset2 = pd.concat([park_scrape_dataset2, word_counts_df], axis=1)
park_scrape_dataset2.head

<bound method NDFrame.head of        park                                                url                               url_stem  access_page  access_page
0      abli                 /abli/planyourvisit/lincolnbio.htm                             lincolnbio            0            0
1      abli                      /abli/planyourvisit/hours.htm                                  hours            0            0
2      abli               /abli/planyourvisit/boyhood-home.htm                           boyhood-home            0            0
3      abli                  /abli/planyourvisit/basicinfo.htm                              basicinfo            0            0
4      abli                 /abli/planyourvisit/directions.htm                             directions            0            0
...     ...                                                ...                                    ...          ...          ...
25355  zion       /zion/planyourvisit/laverkin-creek-trail.htm            

In [12]:
park_word_counts = park_scrape_dataset2.groupby('park')['access_page'].sum().reset_index()
# Display the resulting DataFrame
print(park_word_counts.head())

   park  access_page  access_page
0  abli            3            3
1  acad            3            3
2  adam            2            2
3  afam            0            0
4  afbg            1            1


In [15]:
# Define parking words
parking_words = [
    "parking lot", "parking", "pull off", "pulloff", "pull-off", "total space", "parking space", "oversized space",
"garage", "parking spot", "parking tag", "vehicle space", "parking area"]

# Function to count accessibility words in text
def count_parking_words(text):
    word_counts = {word: text.lower().count(word) for word in parking_words}
    return word_counts

# Apply the function to 'url_stem' column and convert to DataFrame
word_counts_df = park_scrape_dataset2['url_stem'].apply(count_parking_words).apply(pd.Series)
word_counts_df['parking_page'] = word_counts_df.apply(lambda row: 1 if row.any() >= 1 else 0, axis=1)
word_counts_df = word_counts_df[['parking_page']]

# Display the resulting DataFrame
print(word_counts_df.head())
park_scrape_dataset2 = pd.concat([park_scrape_dataset2, word_counts_df], axis=1)
park_scrape_dataset2.head

   parking_page
0             0
1             0
2             0
3             0
4             0


<bound method NDFrame.head of        park                                                url                               url_stem  access_page  access_page  parking_page
0      abli                 /abli/planyourvisit/lincolnbio.htm                             lincolnbio            0            0             0
1      abli                      /abli/planyourvisit/hours.htm                                  hours            0            0             0
2      abli               /abli/planyourvisit/boyhood-home.htm                           boyhood-home            0            0             0
3      abli                  /abli/planyourvisit/basicinfo.htm                              basicinfo            0            0             0
4      abli                 /abli/planyourvisit/directions.htm                             directions            0            0             0
...     ...                                                ...                                    ...          ...    

In [16]:
park_word_counts = park_scrape_dataset2.groupby('park')['parking_page'].sum().reset_index()
# Display the resulting DataFrame
print(park_word_counts.head())

   park  parking_page
0  abli             0
1  acad             0
2  adam             0
3  afam             0
4  afbg             0


In [ ]:
newclusters = pd.merge(clusterparks, parkingpages, on='park', how='left')
newclusters.head()